In [106]:
import psycopg2 as ps2
import random
from decimal import Decimal

In [2]:
connection = None

def getDBCursor():
    global connection
    if (connection == None) or (connection.closed == 0):
        connection = ps2.connect(dbname="testdb", user="postgres", password="apoorvmathur", host="127.0.0.1", port=5432)
        connection.autocommit = True
    cursor = connection.cursor()
    return cursor

In [3]:
def authenticate(user, password):
    cursor = getDBCursor()
    cursor.execute("SELECT * FROM freebie.Users WHERE username = %(username)s", {"username":user})
    result = cursor.fetchone()
    cursor.close()
    if result:
        if result[1] == password:
            return True
    return False

In [10]:
authenticate("apoorvmathur", "apoorv")

True

In [106]:
import random

In [107]:
"'%030x' % random.randrange(16**30)"

"'%030x' % random.randrange(16**30)"

In [108]:
print('%030x' % random.randrange(16**30))

16497a66b56bed920213ccecd1febc


In [4]:
def generateToken(user):
    token = '%030x' % random.randrange(16**30)
    cursor = getDBCursor()
    cursor.execute("INSERT INTO freebie.sessions VALUES (%(token)s, %(username)s) ON CONFLICT (username) DO UPDATE SET token = Excluded.token", 
                   {"token":token, "username":user})
    return token

In [11]:
generateToken("apoorvmathur")

'ab7064057339f6a15ed5bb6b8b3b68'

In [5]:
def verify(user, token):
    cursor = getDBCursor()
    cursor.execute("SELECT * FROM freebie.Sessions where username = %(username)s", {"username":user})
    result = cursor.fetchone()
    cursor.close()
    if result:
        if result[0] == token:
            return True
    return False

In [19]:
verify("apoorvmathur", "94a1e578dcc9604664de8d286427eb")

True

In [6]:
    def addRequest(requestData):
        cursor = getDBCursor()
        query = """INSERT INTO freebie.Requests (agent_name,category,account_phone,sales_date,activation_date,sales_amount,
        prev_sales,discount_amount,discount_percent,alternate_number,discussion_details,status)
        VALUES (%(agent_name)s,%(category)s,%(account_phone)s,%(sales_date)s,%(activation_date)s,%(sales_amount)s,
        %(prev_sales)s,%(discount_amount)s,%(discount_percent)s,%(alternate_number)s,%(discussion_details)s,%(status)s)"""
        cursor.execute(query,requestData)
        cursor.close()
    

In [108]:
request_data = {
    "agent_name":"Amol",
    "category":"Real Estate",
    "account_phone":7906518368,
    "sales_date":"2018-06-11",
    "activation_date":"2018-06-11",
    "sales_amount":10000,
    "prev_sales":10000,
    "discount_amount":10000,
    "discount_percent":18.4,
    "alternate_number":"9458260880",
    "discussion_details":"None",
    "status":"Pending"
}

In [109]:
addRequest(requestData=request_data)

In [60]:
def updateRequest(requestId, status):
    cursor = getDBCursor()
    cursor.execute("UPDATE freebie.Requests SET status = %(status)s WHERE id = %(id)s", {"status":status, "id":requestId})
    cursor.close()

In [24]:
updateRequest(4, "Approved")

In [101]:
def getRequests(userId):
    cursor = getDBCursor()
    cursor.execute("SELECT userlevel FROM Freebie.Users where username = %(username)s", {"username":userId})
    user = cursor.fetchone()
    print(user)
    level = user[0]
    if level == 1:
        print("Level: 1")
        cursor.execute("""select * from freebie.requests a 
            left join freebie.agents b
            on a.agent_name = b.agent
            where a.status = 'Pending'
            and b.team_lead = %(username)s
            and discount_percent <= %(discount)s""", {"username":userId, "discount":5})
    if level == 2:
        print("Level: 2")
        cursor.execute("""select * from freebie.requests a 
            where a.status = 'Pending'
            and a.discount_percent > %(discount_low)s
            and a.discount_percent <= %(discount_high)s""", {"discount_low":5, "discount_high":10})
    if level == 3:
        print("Level: 3")
        cursor.execute("""select * from freebie.requests a 
            where a.status = 'Pending'
            and a.discount_percent > %(discount_low)s
            and a.discount_percent <= %(discount_high)s""", {"discount_low":10, "discount_high":100})
    result = cursor.fetchall()
    cursor.close()
    print(result)
    json_result = convertToJSON(result)
    return json_result

In [57]:
cursor = getDBCursor()
cursor.execute("""select * from freebie.requests a 
    left join freebie.agents b
    on a.agent_name = b.agent
    where a.status = 'Pending'
    and b.team_lead = %(username)s
    and discount_percent <= %(discount)s;""", {"username":"apoorv1", "discount":5})


In [58]:
result = cursor.fetchall()

In [43]:
cursor.execute("SELECT userlevel FROM Freebie.Users where username = %(username)s", {"username":"apoorv"})

In [44]:
user = cursor.fetchone()

In [47]:
level = user[0]

In [117]:
getRequests("apoorv2")

(3,)
Level: 3
[(1, 'jkhdah', 'Real Estate', 7127087, datetime.date(2018, 11, 11), datetime.date(2018, 11, 11), 973921, 9879, 98798, Decimal('10.144354624245704'), '', 'jkfgkasjddjfhkjs', 'Pending'), (2, 'apoorv', 'Real Estate', 7127087, datetime.date(2018, 11, 11), datetime.date(2018, 11, 11), 973921, 9879, 98798, Decimal('10.144354624245704'), '', 'jkfgkasjddjfhkjs', 'Pending')]


[{'account_phone': 7127087,
  'activation_date': datetime.date(2018, 11, 11),
  'agent_name': 'jkhdah',
  'alternate_number': '',
  'category': 'Real Estate',
  'discount_amount': 98798,
  'discount_percent': 10.144354624245704,
  'discussion_details': 'jkfgkasjddjfhkjs',
  'id': 1,
  'prev_sales': 9879,
  'sales_amount': 973921,
  'sales_date': datetime.date(2018, 11, 11),
  'status': 'Pending'},
 {'account_phone': 7127087,
  'activation_date': datetime.date(2018, 11, 11),
  'agent_name': 'apoorv',
  'alternate_number': '',
  'category': 'Real Estate',
  'discount_amount': 98798,
  'discount_percent': 10.144354624245704,
  'discussion_details': 'jkfgkasjddjfhkjs',
  'id': 2,
  'prev_sales': 9879,
  'sales_amount': 973921,
  'sales_date': datetime.date(2018, 11, 11),
  'status': 'Pending'}]

In [113]:
def convertToJSON(result_set):
    outJSON = []
    for result in result_set:
        row = {
            "id":result[0],
            "agent_name":result[1],
            "category":result[2],
            "account_phone":result[3],
            "sales_date":result[4],
            "activation_date":result[5],
            "sales_amount":result[6],
            "prev_sales":result[7],
            "discount_amount":result[8],
            "discount_percent":float(result[9]),
            "alternate_number":result[10],
            "discussion_details":result[11],
            "status":result[12]
        }
        outJSON.append(row)
    return outJSON

In [95]:
convertToJSON(temp)

[{'account_phone': 7906518368,
  'activation_date': datetime.date(2018, 6, 11),
  'agent_name': 'Amol',
  'alternate_number': '9458260880',
  'category': 'Real Estate',
  'discount_amount': 10000,
  'discount_percent': Decimal('18'),
  'discussion_details': 'None',
  'id': 15,
  'prev_sales': 10000,
  'sales_amount': 10000,
  'sales_date': datetime.date(2018, 6, 11),
  'status': 'Pending'}]

In [112]:
float(Decimal('5.18'))

5.18